In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt
from pandas_profiling import ProfileReport
from statsmodels.formula.api import ols
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Lasso
import pickle as pkl
plt.style.use('fivethirtyeight')


pd.set_option('display.max_columns', 500)

In [3]:
m = pd.read_csv('student_mat.csv', sep=';')
p = pd.read_csv('student_por.csv', sep=';')
p_orig = pd.read_csv('student_por.csv', sep=';')
m_orig = pd.read_csv('student_mat.csv', sep=';')

In [4]:
def clean_up(data):
    data.address = data.address.apply(lambda x: 'Urban' if x == 'U' else 'Rural')
    data = pd.get_dummies(data=data, columns=['address'], prefix='address_type', drop_first=True)
    data.famsize = data.famsize.apply(lambda x: 'BIG_FAMILY' if x == 'GT3' else 'SMALL_FAMILY') 
    data = pd.get_dummies(data=data, columns=['famsize'])
    data = data.drop(columns='famsize_SMALL_FAMILY')
    data.Pstatus = data.Pstatus.apply(lambda x: 'Together' if x == 'T' else 'Apart') 
    data = pd.get_dummies(data=data, columns=['Pstatus'], drop_first=True)
    data = pd.get_dummies(data=data, columns=['Mjob', 'Fjob'])
    data = data.drop(columns=['Mjob_other', 'Fjob_other'])
    data = pd.get_dummies(data=data, columns=['reason'])
    data = data.drop(columns=['reason_other'])
    data.schoolsup = data.schoolsup.apply(lambda x: 1 if x == 'yes' else 0) 
    data.famsup = data.famsup.apply(lambda x: 1 if x == 'yes' else 0) 
    data['tutor'] = data.paid.apply(lambda x: 1 if x == 'yes' else 0)
    data = data.drop(columns='paid')
    data.activities = data.activities.apply(lambda x: 1 if x == 'yes' else 0) 
    data.nursery = data.nursery.apply(lambda x: 1 if x == 'yes' else 0) 
    data.higher = data.higher.apply(lambda x: 1 if x == 'yes' else 0) 
    data.internet = data.internet.apply(lambda x: 1 if x == 'yes' else 0) 
    data.romantic = data.romantic.apply(lambda x: 1 if x == 'yes' else 0) 
    return data 

In [5]:
p = clean_up(p)
m = clean_up(m)

In [6]:
p

,school,sex,age,Medu,Fedu,guardian,traveltime,studytime,failures,schoolsup,famsup,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,address_type_Urban,famsize_BIG_FAMILY,Pstatus_Together,Mjob_at_home,Mjob_health,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_reputation,tutor
0,GP,F,18,4,4,mother,2,2,0,1,0,0,1,1,0,0,4,3,4,1,1,3,4,0,11,11,1,1,0,1,0,0,0,0,0,0,1,1,0,0,0
1,GP,F,17,1,1,father,1,2,0,0,1,0,0,1,1,0,5,3,3,1,1,3,2,9,11,11,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0
2,GP,F,15,1,1,mother,1,2,0,1,0,0,1,1,1,0,4,3,2,2,3,3,6,12,13,12,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0
3,GP,F,15,4,2,mother,1,3,0,0,1,1,1,1,1,1,3,2,2,1,1,5,0,14,14,14,1,1,1,0,1,0,0,0,0,1,0,0,1,0,0
4,GP,F,16,3,3,father,1,2,0,0,1,0,1,1,0,0,4,3,2,1,2,5,0,11,13,13,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,2,3,mother,1,3,1,0,0,1,0,1,1,0,5,4,2,1,2,5,4,10,11,10,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0
645,MS,F,18,3,1,mother,1,2,0,0,1,0,1,1,1,0,4,3,4,1,1,1,4,15,15,16,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0
646,MS,F,18,1,1,mother,2,2,0,0,0,1,1,1,0,0,1,1,1,1,1,5,6,11,12,9,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0
647,MS,M,17,3,1,mother,2,1,0,0,0,0,0,1,1,0,2,4,5,3,4,2,6,10,10,10,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0


In [7]:
m

,school,sex,age,Medu,Fedu,guardian,traveltime,studytime,failures,schoolsup,famsup,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,address_type_Urban,famsize_BIG_FAMILY,Pstatus_Together,Mjob_at_home,Mjob_health,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_reputation,tutor
0,GP,F,18,4,4,mother,2,2,0,1,0,0,1,1,0,0,4,3,4,1,1,3,6,5,6,6,1,1,0,1,0,0,0,0,0,0,1,1,0,0,0
1,GP,F,17,1,1,father,1,2,0,0,1,0,0,1,1,0,5,3,3,1,1,3,4,5,5,6,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0
2,GP,F,15,1,1,mother,1,2,3,1,0,0,1,1,1,0,4,3,2,2,3,3,10,7,8,10,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1
3,GP,F,15,4,2,mother,1,3,0,0,1,1,1,1,1,1,3,2,2,1,1,5,2,15,14,15,1,1,1,0,1,0,0,0,0,1,0,0,1,0,1
4,GP,F,16,3,3,father,1,2,0,0,1,0,1,1,0,0,4,3,2,1,2,5,4,6,10,10,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,2,2,other,1,2,2,0,1,0,1,1,0,0,5,5,4,4,5,4,11,9,9,9,1,0,0,0,0,1,0,0,0,1,0,1,0,0,1
391,MS,M,17,3,1,mother,2,1,0,0,0,0,0,1,1,0,2,4,5,3,4,2,3,14,16,16,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0
392,MS,M,21,1,1,other,1,1,3,0,0,0,0,1,0,0,5,5,3,3,3,3,3,10,8,7,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0
393,MS,M,18,3,2,mother,3,1,0,0,0,0,0,1,1,0,4,4,1,3,4,5,0,11,12,10,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0


In [8]:
features = [x for x in p.columns if (x != 'school') and x != 'age' and x != 'guardian' and x != 'G1' and x != 'G2' and x!='G3' and x!= 'sex']
features

['Medu',
 'Fedu',
 'traveltime',
 'studytime',
 'failures',
 'schoolsup',
 'famsup',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic',
 'famrel',
 'freetime',
 'goout',
 'Dalc',
 'Walc',
 'health',
 'absences',
 'address_type_Urban',
 'famsize_BIG_FAMILY',
 'Pstatus_Together',
 'Mjob_at_home',
 'Mjob_health',
 'Mjob_services',
 'Mjob_teacher',
 'Fjob_at_home',
 'Fjob_health',
 'Fjob_services',
 'Fjob_teacher',
 'reason_course',
 'reason_home',
 'reason_reputation',
 'tutor']

In [9]:
p_feature = p[features]
test_p_feature = p_feature
m_feature = m[features]

In [10]:
test_p_feature['Testcol'] =  ((5/4)*p_feature.Medu)+((5/4)*p_feature.Fedu)+(5*p_feature.activities)+(5*p_feature.higher)-p_feature.Dalc-p_feature.Walc


m_orig.to_csv('MathClassOriginal.csv')
p_orig.to_csv('PortugeseClassOriginal.csv')
m.to_csv('MathClassCleaned.csv')
p.to_csv('PortugeseClassCleaned.csv')
p_feature.to_csv('pFeatures.csv')
m_feature.to_csv('mFeatures.csv')
test_p_feature.to_csv('testP.csv')


<ipython-input-10-616192e095fb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_p_feature['Testcol'] =  ((5/4)*p_feature.Medu)+((5/4)*p_feature.Fedu)+(5*p_feature.activities)+(5*p_feature.higher)-p_feature.Dalc-p_feature.Walc


In [12]:
pkl.dump(features,open('features.pkl','wb'))